In [1]:
import os
import numpy as np
os.getcwd()

'C:\\_A\\py_examples\\turtle'

In [2]:
def maze_template1(size=10):
    len = size
    arr = np.zeros((len,len))
    return arr

print (mt := maze_template1())

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [83]:
import numpy as np
from random import randint
from enum import Enum
import random

class Direction(Enum):
    UP = 1
    DOWN = 2
    LEFT = 3
    RIGHT = 4
    
class Maze:
    def __init__(self, size=10, start=(0,0), goal=None, forward_probability=0.4,seed=42):
        self._found_cells = np.zeros((size,size))
        self._explored_cells = np.zeros((size,size))
        self._grid = -1 * np.ones( (2*size-1, 2*size-1))
        self._size=size
        self._start=start
        self._forward_probability=forward_probability
        self._seed=seed
        self._pathes = []
        self._current_path = [self._start]
        self._goal = goal
        random.seed(self._seed)
        if self._goal is None:
            self._goal = (size - 1, size - 1)
        f = self._forward_probability
        b = 1 - self._forward_probability
        self._move_weights = {
            Direction.UP: b,
            Direction.LEFT: b,
            Direction.DOWN: f,
            Direction.RIGHT: f}
        self._move_delta = {
            Direction.UP: (-1,0),
            Direction.DOWN: (1,0),
            Direction.LEFT: (0,-1),
            Direction.RIGHT: (0,1) }

    def _move(self):
        pass

    def _position(self):
        return self._current_path[-1]

    def _move_step(self, position, delta):
        arr = np.array(position) + np.array(delta)
        return (arr[0], arr[1])

    def _is_valid(self,position):
        r,c = position
        if 0 <= r and r < self._size and 0 <= c and c < self._size:
            return True
        else:
            return False
    def _is_available(self,position):
        r,c = position
        return self._found_cells[r,c] == 0

    def _allow_moves(self):
        moves = []
        pos = self._position()
        for dir in Direction:
            weight = self._move_weights[dir]
            md = self._move_delta[dir]
            new_pos = self._move_step(pos,md)
            if self._is_valid(new_pos) and self._is_available(new_pos):
                moves.append((weight, new_pos))
        return moves

    def _weighted_choice(self, moves):
        weights = list(map(lambda x : x[0], moves))
        total = sum(weights)
        accum = 0.0
        x = random.random()
        for w, m in moves:
            accum = accum + w/total
            if accum >= x:
                return m 
        print(f'WARNING: _weighted_choice returns None for moves = {moves}')
        return None

    def _next_move(self):
        allowed = self._allow_moves()
        if not allowed:
            return None
        new_pos = self._weighted_choice(allowed)
        print(f'Next Move: {new_pos}')
        return new_pos

    def _choose_new_start(self):
        for i in range(10 * self._size * self._size):
            path_index = randint(0, len(self._pathes) - 1)
            path = self._pathes[path_index]
            step_index = randint(0, len(path) - 1)
            new_start = path[step_index]
            r,c = new_start
            if self._explored_cells[r,c] == 0:
                self._explored_cells[r,c] = 1
                return new_start

        return None
            

    def search(self):
        while True: ## Outer loop thru possible branches of search tree

            while True: ## Inner loop thru moves from a given start position

                new_pos = self._next_move()
                if new_pos is None:
                    break

                self._current_path.append(new_pos)
                r,c = new_pos
                self._found_cells[r,c] = 1

            print(f'Current path ended at {self._position()}')

            self._pathes.append(self._current_path)
            self._current_path = []
            new_pos = self._choose_new_start()
            if new_pos is None:
                break
            self._current_path.append(new_pos)
            print(f'Starting new path at {self._position()}')

    def _mark_grid(self, pair):
        from1 = np.array(pair[0])
        to = np.array(pair[1])
        delta = to - from1
        from2 = 2 * from1
        from2p1 = from2 + delta
        from2p2 = from2p1 + delta
        marks = list ( (from2, from2p1, from2p2) )
        for mark in marks:
            self._grid[mark[0],mark[1]] = 0
        

    def make_grid(self):
        for path in self._pathes:
            for start in range(len(path) - 1):
                pair = path[start : start+2]
                self._mark_grid(pair)
                
                       
                
            
        
        

In [84]:
m = Maze(size=7)

moves = m._allow_moves()

m._weighted_choice(moves)

(0, 1)

In [85]:
m.search()

Next Move: (1, 0)
Next Move: (0, 0)
Next Move: (0, 1)
Next Move: (0, 2)
Next Move: (0, 3)
Next Move: (0, 4)
Next Move: (1, 4)
Next Move: (1, 3)
Next Move: (2, 3)
Next Move: (3, 3)
Next Move: (3, 2)
Next Move: (2, 2)
Next Move: (1, 2)
Next Move: (1, 1)
Next Move: (2, 1)
Next Move: (3, 1)
Next Move: (3, 0)
Next Move: (4, 0)
Next Move: (5, 0)
Next Move: (5, 1)
Next Move: (6, 1)
Next Move: (6, 0)
Current path ended at (6, 0)
Starting new path at (0, 4)
Next Move: (0, 5)
Next Move: (1, 5)
Next Move: (2, 5)
Next Move: (3, 5)
Next Move: (3, 6)
Next Move: (4, 6)
Next Move: (4, 5)
Next Move: (4, 4)
Next Move: (5, 4)
Next Move: (5, 5)
Next Move: (6, 5)
Next Move: (6, 4)
Next Move: (6, 3)
Next Move: (6, 2)
Next Move: (5, 2)
Next Move: (4, 2)
Next Move: (4, 3)
Next Move: (5, 3)
Current path ended at (5, 3)
Starting new path at (2, 3)
Next Move: (2, 4)
Next Move: (3, 4)
Current path ended at (3, 4)
Starting new path at (2, 2)
Current path ended at (2, 2)
Starting new path at (2, 4)
Current path end

In [86]:
print(m._found_cells)

[[1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]]


In [87]:
print(m._explored_cells)

[[1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]]


In [88]:
np.sum(m._explored_cells)

47.0

In [89]:
f = (0,0)
t = (0,1)
ft = [f,t]
print(ft)
m._mark_grid(ft)

[(0, 0), (0, 1)]


In [90]:
m._grid

array([[ 0.,  0.,  0., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.]])

In [91]:
m.make_grid()

In [92]:
m._grid

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1., -1., -1., -1., -1., -1., -1.,  0., -1.,  0., -1.,  0.],
       [ 0., -1.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0., -1.,  0.],
       [-1., -1.,  0., -1.,  0., -1.,  0., -1., -1., -1.,  0., -1.,  0.],
       [ 0., -1.,  0., -1.,  0., -1.,  0.,  0.,  0., -1.,  0., -1.,  0.],
       [ 0., -1.,  0., -1.,  0., -1.,  0., -1.,  0., -1.,  0., -1., -1.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0., -1.,  0.,  0.,  0.],
       [ 0., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1., -1., -1.,  0., -1.,  0., -1.,  0., -1., -1., -1., -1.],
       [ 0.,  0.,  0., -1.,  0., -1.,  0., -1.,  0.,  0.,  0., -1.,  0.],
       [-1., -1.,  0., -1.,  0., -1., -1., -1., -1., -1.,  0., -1.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [93]:
np.savetxt('data/output.csv', m._grid, delimiter=",")


In [94]:
arr = np.loadtxt('data/output.csv',delimiter=",")


In [95]:
arr

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1., -1., -1., -1., -1., -1., -1.,  0., -1.,  0., -1.,  0.],
       [ 0., -1.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0., -1.,  0.],
       [-1., -1.,  0., -1.,  0., -1.,  0., -1., -1., -1.,  0., -1.,  0.],
       [ 0., -1.,  0., -1.,  0., -1.,  0.,  0.,  0., -1.,  0., -1.,  0.],
       [ 0., -1.,  0., -1.,  0., -1.,  0., -1.,  0., -1.,  0., -1., -1.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0., -1.,  0.,  0.,  0.],
       [ 0., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1., -1., -1.,  0., -1.,  0., -1.,  0., -1., -1., -1., -1.],
       [ 0.,  0.,  0., -1.,  0., -1.,  0., -1.,  0.,  0.,  0., -1.,  0.],
       [-1., -1.,  0., -1.,  0., -1., -1., -1., -1., -1.,  0., -1.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])